In [1]:
import numpy as np
import pandas as pd
import os
import math

from scipy import stats
from scipy.stats import norm 

import math
import sklearn
import sklearn.preprocessing
import datetime
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore')

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
path = "input/"

# original data : data 
data =  pd.read_csv(path + "demential_data.csv", parse_dates=['summary_date']) # 애초에 datatime 유형의 데이터를 읽어 올 때부터 형변환하기
data

,EMAIL,summary_date,activity_average_met,activity_cal_active,activity_cal_total,activity_class_5min,activity_daily_movement,activity_high,activity_inactive,activity_inactivity_alerts,...,sleep_temperature_deviation,sleep_temperature_trend_deviation,timezone,sleep_total,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8),CONVERT(sleep_hr_5min USING utf8),CONVERT(sleep_hypnogram_5min USING utf8),CONVERT(sleep_rmssd_5min USING utf8),DIAG_NM
0,nia+404@rowan.kr,2020-11-27,1.71875,730,2944,...,14346,0,417,0,...,-0.12,99.99,NaN,\r,2/1/1/1/1/1/2/2/1/1/1/1/1/1/2/2/2/3/2/2/2/2/2/...,0.9/0.9/1.4/1.9/1.1/0.9/0.9/1.1/1.3/1/0.9/1.1/...,0/73/73/73/72/71/70/71/71/71/70/70/73/72/74/74...,4/2/4/3/3/1/2/2/2/2/2/2/3/3/3/4/4/3/2/2/2/2/2/...,0/10/10/10/11/11/10/12/18/13/14/12/10/10/18/17...,CN
1,nia+404@rowan.kr,2020-11-28,1.40625,342,2449,...,6352,0,473,0,...,-0.32,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/2/2/2/2/...,1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,69/70/69/69/70/72/71/72/70/69/69/69/68/68/63/6...,2/4/2/2/2/2/3/1/2/2/4/4/2/2/2/2/2/2/2/2/2/2/4/...,23/23/26/24/18/13/15/14/17/20/24/30/23/25/22/1...,CN
2,nia+404@rowan.kr,2020-11-29,1.46875,401,2544,...,7297,0,586,0,...,0.07,99.99,NaN,\r,1/1/1/1/1/1/1/2/1/1/1/1/2/2/2/2/2/1/1/1/1/1/2/...,1.1/1.1/1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/74/73/73/74/74/74/71/71/70/70/69/70/68/66/69...,4/2/4/4/1/1/1/4/4/4/4/4/4/4/2/3/4/2/2/4/2/2/2/...,0/11/14/20/13/14/14/16/27/29/27/20/19/19/14/12...,CN
3,nia+404@rowan.kr,2020-11-30,0.34375,27,1850,...,491,0,176,0,...,-0.41,99.99,NaN,\r,2/1/2/2/1/2/1/1/2/1/1/1/1/1/2/1/1/1/1/1/2/2/2/...,0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/...,73/70/71/72/75/75/73/70/70/70/67/63/63/63/63/6...,4/4/4/4/3/3/3/2/4/4/4/2/2/2/2/2/2/2/2/4/2/2/2/...,24/28/19/17/12/10/17/20/23/23/25/31/26/25/34/3...,CN
4,nia+404@rowan.kr,2020-12-01,1.46875,333,2518,...,5861,0,646,0,...,-0.27,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/3/3/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/0/0/0/0/0/0/0/69/69/71/69/65/66/64/64/65/66/...,4/4/4/4/4/4/4/4/4/4/4/2/2/2/2/3/3/2/4/4/4/2/2/...,0/0/0/0/0/0/0/0/21/22/26/23/19/29/22/17/14/13/...,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,nia+206@rowan.kr,2020-12-22,1.34375,227,2316,...,3863,3,735,2,...,-0.16,99.99,NaN,\r,2/1/1/2/2/1/1/1/1/2/2/3/2/1/1/1/1/1/1/1/1/1/1/...,1.1/1.4/1.2/0.9/1.2/1.1/0.9/0.9/0.9/1/0.9/0.9/...,0/54/54/54/55/56/0/0/55/52/52/53/54/54/56/57/6...,4/4/4/2/2/2/2/4/2/2/2/1/1/1/1/4/2/2/3/3/3/3/3/...,0/35/39/28/26/41/0/0/52/31/27/30/21/22/27/40/3...,CN
9323,nia+206@rowan.kr,2020-12-24,1.34375,249,2351,...,4411,1,780,4,...,-0.09,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/2/3/2/2/2/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,68/66/67/67/68/69/69/70/71/71/71/69/72/70/70/7...,4/4/2/2/2/1/1/1/1/1/1/2/2/3/3/2/2/2/2/2/3/4/2/...,26/16/19/18/16/18/19/17/15/16/15/13/11/13/13/1...,CN
9324,nia+206@rowan.kr,2020-12-26,1.53125,570,2682,...,11057,1,518,1,...,0.41,99.99,NaN,\r,1/1/1/1/1/1/2/2/2/2/2/2/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/1.1/0.9/0.9/1.1/0.9/0.9/0.9/0....,0/0/65/65/66/67/69/72/73/72/73/74/73/74/75/75/...,4/4/2/2/2/2/2/2/2/2/2/2/4/4/4/4/2/3/3/3/2/2/4/...,0/0/13/12/12/13/12/12/12/16/13/12/13/17/13/18/...,CN
9325,nia+206@rowan.kr,2020-12-27,1.34375,295,2331,...,5135,4,578,0,...,0.27,99.99,NaN,\r,1/1/1/2/3/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,0/0/0/64/64/64/64/65/65/66/66/66/66/68/68/68/6...,4/4/4/4/2/2/2/2/2/2/2/2/2/3/3/3/3/2/4/4/2/2/2/...,0/0/0/14/17/18/14/14/17/15/17/12/17/12/13/12/1...,CN


In [3]:
# column의 수가 많기 때문에 효율적으로 관리하기 위해서 
# column의 이름들을 묶어서 진행합시다

drop_cols = ['timezone', 'EMAIL',
             'activity_class_5min', 'activity_met_1min', 
             'sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min',
             'sleep_is_longest', 'sleep_temperature_trend_deviation', 'sleep_total', 'sleep_temperature_deviation']
convert_cols = data.iloc[:, 60:66].columns
data_convert = data[convert_cols].copy()
data_convert

,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8),CONVERT(sleep_hr_5min USING utf8),CONVERT(sleep_hypnogram_5min USING utf8),CONVERT(sleep_rmssd_5min USING utf8),DIAG_NM
0,2/1/1/1/1/1/2/2/1/1/1/1/1/1/2/2/2/3/2/2/2/2/2/...,0.9/0.9/1.4/1.9/1.1/0.9/0.9/1.1/1.3/1/0.9/1.1/...,0/73/73/73/72/71/70/71/71/71/70/70/73/72/74/74...,4/2/4/3/3/1/2/2/2/2/2/2/3/3/3/4/4/3/2/2/2/2/2/...,0/10/10/10/11/11/10/12/18/13/14/12/10/10/18/17...,CN
1,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/2/2/2/2/...,1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,69/70/69/69/70/72/71/72/70/69/69/69/68/68/63/6...,2/4/2/2/2/2/3/1/2/2/4/4/2/2/2/2/2/2/2/2/2/2/4/...,23/23/26/24/18/13/15/14/17/20/24/30/23/25/22/1...,CN
2,1/1/1/1/1/1/1/2/1/1/1/1/2/2/2/2/2/1/1/1/1/1/2/...,1.1/1.1/1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/74/73/73/74/74/74/71/71/70/70/69/70/68/66/69...,4/2/4/4/1/1/1/4/4/4/4/4/4/4/2/3/4/2/2/4/2/2/2/...,0/11/14/20/13/14/14/16/27/29/27/20/19/19/14/12...,CN
3,2/1/2/2/1/2/1/1/2/1/1/1/1/1/2/1/1/1/1/1/2/2/2/...,0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/...,73/70/71/72/75/75/73/70/70/70/67/63/63/63/63/6...,4/4/4/4/3/3/3/2/4/4/4/2/2/2/2/2/2/2/2/4/2/2/2/...,24/28/19/17/12/10/17/20/23/23/25/31/26/25/34/3...,CN
4,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/3/3/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/0/0/0/0/0/0/0/69/69/71/69/65/66/64/64/65/66/...,4/4/4/4/4/4/4/4/4/4/4/2/2/2/2/3/3/2/4/4/4/2/2/...,0/0/0/0/0/0/0/0/21/22/26/23/19/29/22/17/14/13/...,CN
...,...,...,...,...,...,...
9322,2/1/1/2/2/1/1/1/1/2/2/3/2/1/1/1/1/1/1/1/1/1/1/...,1.1/1.4/1.2/0.9/1.2/1.1/0.9/0.9/0.9/1/0.9/0.9/...,0/54/54/54/55/56/0/0/55/52/52/53/54/54/56/57/6...,4/4/4/2/2/2/2/4/2/2/2/1/1/1/1/4/2/2/3/3/3/3/3/...,0/35/39/28/26/41/0/0/52/31/27/30/21/22/27/40/3...,CN
9323,1/1/1/1/1/1/1/1/1/1/2/3/2/2/2/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,68/66/67/67/68/69/69/70/71/71/71/69/72/70/70/7...,4/4/2/2/2/1/1/1/1/1/1/2/2/3/3/2/2/2/2/2/3/4/2/...,26/16/19/18/16/18/19/17/15/16/15/13/11/13/13/1...,CN
9324,1/1/1/1/1/1/2/2/2/2/2/2/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/1.1/0.9/0.9/1.1/0.9/0.9/0.9/0....,0/0/65/65/66/67/69/72/73/72/73/74/73/74/75/75/...,4/4/2/2/2/2/2/2/2/2/2/2/4/4/4/4/2/3/3/3/2/2/4/...,0/0/13/12/12/13/12/12/12/16/13/12/13/17/13/18/...,CN
9325,1/1/1/2/3/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,0/0/0/64/64/64/64/65/65/66/66/66/66/68/68/68/6...,4/4/4/4/2/2/2/2/2/2/2/2/2/3/3/3/3/2/4/4/2/2/2/...,0/0/0/14/17/18/14/14/17/15/17/12/17/12/13/12/1...,CN


In [4]:
signal_list = list(data_convert['CONVERT(activity_class_5min USING utf8)'].str.split('/'))
signal_activity_class_5min = pd.DataFrame(signal_list).drop(288, axis=1).astype('float')
y_df = data_convert['DIAG_NM'] 
signal_activity_class_5min = pd.concat([signal_activity_class_5min, y_df], axis=1)
signal_activity_class_5min

,0,1,2,3,4,5,6,7,8,9,...,279,280,281,282,283,284,285,286,287,DIAG_NM
0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,...,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
3,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9323,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9324,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9325,1.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN


In [5]:
signal_list = list(data_convert['CONVERT(activity_met_1min USING utf8)'].str.split('/'))
signal_activity_met_1min = pd.DataFrame(signal_list).drop(1440, axis=1).astype('float')
signal_activity_met_1min = pd.concat([signal_activity_met_1min, y_df], axis=1)
signal_activity_met_1min

,0,1,2,3,4,5,6,7,8,9,...,1431,1432,1433,1434,1435,1436,1437,1438,1439,DIAG_NM
0,0.9,0.9,1.4,1.9,1.1,0.9,0.9,1.1,1.3,1.0,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
1,1.2,1.1,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
2,1.1,1.1,1.2,1.1,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,1.1,1.4,1.2,0.9,1.2,1.1,0.9,0.9,0.9,1.0,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9323,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9324,0.9,0.9,0.9,0.9,1.1,0.9,0.9,1.1,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9325,1.1,0.9,0.9,0.9,1.0,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN


In [6]:
signal_list = list(data_convert['CONVERT(sleep_hr_5min USING utf8)'].str.split('/'))
signal_sleep_hr_5min = pd.DataFrame(signal_list)
signal_sleep_hr_5min

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,181
0,0,73,73,73,72,71,70,71,71,71,...,None,None,None,None,None,None,None,None,None,None
1,69,70,69,69,70,72,71,72,70,69,...,None,None,None,None,None,None,None,None,None,None
2,0,74,73,73,74,74,74,71,71,70,...,None,None,None,None,None,None,None,None,None,None
3,73,70,71,72,75,75,73,70,70,70,...,None,None,None,None,None,None,None,None,None,None
4,0,0,0,0,0,0,0,0,69,69,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0,54,54,54,55,56,0,0,55,52,...,None,None,None,None,None,None,None,None,None,None
9323,68,66,67,67,68,69,69,70,71,71,...,None,None,None,None,None,None,None,None,None,None
9324,0,0,65,65,66,67,69,72,73,72,...,None,None,None,None,None,None,None,None,None,None
9325,0,0,0,64,64,64,64,65,65,66,...,None,None,None,None,None,None,None,None,None,None


In [7]:
signal_sleep_hr_5min = signal_sleep_hr_5min.drop(181, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_hr_5min = signal_sleep_hr_5min.fillna(0) 
signal_sleep_hr_5min = signal_sleep_hr_5min.replace('', 0)
signal_sleep_hr_5min = signal_sleep_hr_5min.replace(' ', 0)
signal_sleep_hr_5min = signal_sleep_hr_5min.astype('float')

In [8]:
signal_sleep_hr_5min  = signal_sleep_hr_5min.transpose()
signal_sleep_hr_5min = pd.concat([signal_sleep_hr_5min, y_df], axis=1)
signal_sleep_hr_5min

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,DIAG_NM
0,0.0,73.0,73.0,73.0,72.0,71.0,70.0,71.0,71.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,69.0,70.0,69.0,69.0,70.0,72.0,71.0,72.0,70.0,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,0.0,74.0,73.0,73.0,74.0,74.0,74.0,71.0,71.0,70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,73.0,70.0,71.0,72.0,75.0,75.0,73.0,70.0,70.0,70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.0,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0.0,54.0,54.0,54.0,55.0,56.0,0.0,0.0,55.0,52.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,68.0,66.0,67.0,67.0,68.0,69.0,69.0,70.0,71.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,0.0,0.0,65.0,65.0,66.0,67.0,69.0,72.0,73.0,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,0.0,0.0,0.0,64.0,64.0,64.0,64.0,65.0,65.0,66.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


In [9]:
signal_list = list(data_convert['CONVERT(sleep_hypnogram_5min USING utf8)'].str.split('/'))
signal_sleep_hypnogram_5min = pd.DataFrame(signal_list)

signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.drop(180, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.fillna(0) 
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.replace('', 0)
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.replace(' ', 0)
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.astype('float')

In [10]:
signal_sleep_hypnogram_5min  = signal_sleep_hypnogram_5min.transpose()
signal_sleep_hypnogram_5min = pd.concat([signal_sleep_hypnogram_5min, y_df], axis=1)
signal_sleep_hypnogram_5min

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,DIAG_NM
0,4.0,2.0,4.0,3.0,3.0,1.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,2.0,4.0,2.0,2.0,2.0,2.0,3.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,4.0,2.0,4.0,4.0,1.0,1.0,1.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,4.0,4.0,4.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,4.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,4.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


In [11]:
signal_list = list(data_convert['CONVERT(sleep_rmssd_5min USING utf8)'].str.split('/'))
signal_sleep_rmssd_5min = pd.DataFrame(signal_list)

signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.drop(181, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.fillna(0) 
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.replace('', 0)
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.replace(' ', 0)
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.astype('float')

In [12]:
signal_sleep_rmssd_5min  = signal_sleep_rmssd_5min.transpose()
signal_sleep_rmssd_5min = pd.concat([signal_sleep_rmssd_5min, y_df], axis=1)
signal_sleep_rmssd_5min

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,DIAG_NM
0,0.0,10.0,10.0,10.0,11.0,11.0,10.0,12.0,18.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,23.0,23.0,26.0,24.0,18.0,13.0,15.0,14.0,17.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,0.0,11.0,14.0,20.0,13.0,14.0,14.0,16.0,27.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,24.0,28.0,19.0,17.0,12.0,10.0,17.0,20.0,23.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0.0,35.0,39.0,28.0,26.0,41.0,0.0,0.0,52.0,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,26.0,16.0,19.0,18.0,16.0,18.0,19.0,17.0,15.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,0.0,0.0,13.0,12.0,12.0,13.0,12.0,12.0,12.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,0.0,0.0,0.0,14.0,17.0,18.0,14.0,14.0,17.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


In [13]:
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV

# y 변수와 X 변수 분할 관리 
input_data_y = y_df.copy()
input_data_X1 = signal_activity_class_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X2 = signal_activity_met_1min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X3 = signal_sleep_hr_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X4 = signal_sleep_hypnogram_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X5 = signal_sleep_rmssd_5min.drop(labels = 'DIAG_NM', axis = 1)

In [14]:
# LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
input_data_y_label = le.fit_transform(list(input_data_y))

print("original gain_label \n", le.classes_)
print("gain_lable label \n", set(input_data_y_label))

original gain_label 
 ['CN' 'Dem' 'MCI']
gain_lable label 
 {0, 1, 2}


In [15]:
X1_train_label, X1_test_label, y1_train_label, y1_test_label = train_test_split(
    input_data_X1, input_data_y_label, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)
X2_train_label, X2_test_label, y2_train_label, y2_test_label = train_test_split(
    input_data_X2, input_data_y_label, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)
X3_train_label, X3_test_label, y3_train_label, y3_test_label = train_test_split(
    input_data_X3, input_data_y_label, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)
X4_train_label, X4_test_label, y4_train_label, y4_test_label = train_test_split(
    input_data_X4, input_data_y_label, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)
X5_train_label, X5_test_label, y5_train_label, y5_test_label = train_test_split(
    input_data_X5, input_data_y_label, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)

In [16]:
cols_convert = data.columns[60:66]
data_convert = data[cols_convert].copy()
data_convert

,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8),CONVERT(sleep_hr_5min USING utf8),CONVERT(sleep_hypnogram_5min USING utf8),CONVERT(sleep_rmssd_5min USING utf8),DIAG_NM
0,2/1/1/1/1/1/2/2/1/1/1/1/1/1/2/2/2/3/2/2/2/2/2/...,0.9/0.9/1.4/1.9/1.1/0.9/0.9/1.1/1.3/1/0.9/1.1/...,0/73/73/73/72/71/70/71/71/71/70/70/73/72/74/74...,4/2/4/3/3/1/2/2/2/2/2/2/3/3/3/4/4/3/2/2/2/2/2/...,0/10/10/10/11/11/10/12/18/13/14/12/10/10/18/17...,CN
1,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/2/2/2/2/...,1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,69/70/69/69/70/72/71/72/70/69/69/69/68/68/63/6...,2/4/2/2/2/2/3/1/2/2/4/4/2/2/2/2/2/2/2/2/2/2/4/...,23/23/26/24/18/13/15/14/17/20/24/30/23/25/22/1...,CN
2,1/1/1/1/1/1/1/2/1/1/1/1/2/2/2/2/2/1/1/1/1/1/2/...,1.1/1.1/1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/74/73/73/74/74/74/71/71/70/70/69/70/68/66/69...,4/2/4/4/1/1/1/4/4/4/4/4/4/4/2/3/4/2/2/4/2/2/2/...,0/11/14/20/13/14/14/16/27/29/27/20/19/19/14/12...,CN
3,2/1/2/2/1/2/1/1/2/1/1/1/1/1/2/1/1/1/1/1/2/2/2/...,0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/...,73/70/71/72/75/75/73/70/70/70/67/63/63/63/63/6...,4/4/4/4/3/3/3/2/4/4/4/2/2/2/2/2/2/2/2/4/2/2/2/...,24/28/19/17/12/10/17/20/23/23/25/31/26/25/34/3...,CN
4,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/3/3/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/0/0/0/0/0/0/0/69/69/71/69/65/66/64/64/65/66/...,4/4/4/4/4/4/4/4/4/4/4/2/2/2/2/3/3/2/4/4/4/2/2/...,0/0/0/0/0/0/0/0/21/22/26/23/19/29/22/17/14/13/...,CN
...,...,...,...,...,...,...
9322,2/1/1/2/2/1/1/1/1/2/2/3/2/1/1/1/1/1/1/1/1/1/1/...,1.1/1.4/1.2/0.9/1.2/1.1/0.9/0.9/0.9/1/0.9/0.9/...,0/54/54/54/55/56/0/0/55/52/52/53/54/54/56/57/6...,4/4/4/2/2/2/2/4/2/2/2/1/1/1/1/4/2/2/3/3/3/3/3/...,0/35/39/28/26/41/0/0/52/31/27/30/21/22/27/40/3...,CN
9323,1/1/1/1/1/1/1/1/1/1/2/3/2/2/2/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,68/66/67/67/68/69/69/70/71/71/71/69/72/70/70/7...,4/4/2/2/2/1/1/1/1/1/1/2/2/3/3/2/2/2/2/2/3/4/2/...,26/16/19/18/16/18/19/17/15/16/15/13/11/13/13/1...,CN
9324,1/1/1/1/1/1/2/2/2/2/2/2/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/1.1/0.9/0.9/1.1/0.9/0.9/0.9/0....,0/0/65/65/66/67/69/72/73/72/73/74/73/74/75/75/...,4/4/2/2/2/2/2/2/2/2/2/2/4/4/4/4/2/3/3/3/2/2/4/...,0/0/13/12/12/13/12/12/12/16/13/12/13/17/13/18/...,CN
9325,1/1/1/2/3/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,0/0/0/64/64/64/64/65/65/66/66/66/66/68/68/68/6...,4/4/4/4/2/2/2/2/2/2/2/2/2/3/3/3/3/2/4/4/2/2/2/...,0/0/0/14/17/18/14/14/17/15/17/12/17/12/13/12/1...,CN


In [17]:
# OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
ohe

OneHotEncoder(sparse=False)

In [18]:
input_data_y.shape
input_data_y.values.reshape(-1,1).shape
ohe.fit(input_data_y.values.reshape(-1,1))
one_hot_encoded = ohe.transform(input_data_y.values.reshape(-1,1))
one_hot_encoded

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [19]:
print(type(ohe.categories_))
ohe.categories_

<class 'list'>


[array(['CN', 'Dem', 'MCI'], dtype=object)]

In [20]:
ohe_df = pd.DataFrame(one_hot_encoded, columns=ohe.categories_[0])
ohe_df

,CN,Dem,MCI
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
9322,1.0,0.0,0.0
9323,1.0,0.0,0.0
9324,1.0,0.0,0.0
9325,1.0,0.0,0.0


In [21]:
input_data_y_ohe = ohe_df.copy()
input_data_y_ohe

,CN,Dem,MCI
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
9322,1.0,0.0,0.0
9323,1.0,0.0,0.0
9324,1.0,0.0,0.0
9325,1.0,0.0,0.0


In [22]:
X1_train_ohe, X1_test_ohe, y1_train_ohe, y1_test_ohe = train_test_split(
    input_data_X1, input_data_y_ohe, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)
X2_train_ohe, X2_test_ohe, y2_train_ohe, y2_test_ohe = train_test_split(
    input_data_X2, input_data_y_ohe, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)
X3_train_ohe, X3_test_ohe, y3_train_ohe, y3_test_ohe = train_test_split(
    input_data_X3, input_data_y_ohe, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)
X4_train_ohe, X4_test_ohe, y4_train_ohe, y4_test_ohe = train_test_split(
    input_data_X4, input_data_y_ohe, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)
X5_train_ohe, X5_test_ohe, y5_train_ohe, y5_test_ohe = train_test_split(
    input_data_X5, input_data_y_ohe, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)

In [23]:
print("X1_label :" ,X1_train_label.shape)
print("X1_ohe :" ,X1_train_ohe.shape)
print("y1_label :" ,y1_train_label.shape)
print("y1_ohe :" ,y1_train_ohe.shape)
# print("X3_label :" ,X3_train_label.shape)
# print("X3_ohe :" ,X3_train_ohe.shape)
# print("y3_label :" ,y3_train_label.shape)
# print("y3_ohe :" ,y3_train_ohe.shape)

X1_label : (7461, 288)
X1_ohe : (7461, 288)
y1_label : (7461,)
y1_ohe : (7461, 3)


# RNN모델 구축 전 데이터 처리

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Activation, SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from keras.layers import Bidirectional
from keras import optimizers
from sklearn.metrics import accuracy_score
from keras.layers import Bidirectional
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping

import tensorflow as tf

input 시 DataFrame이 아닌 배열의 형식으로 넣어야 하므로 ndarray로 변환

In [25]:
X3_train_ohe =  X3_train_ohe.to_numpy()
X3_train_ohe

array([[64., 62., 63., ...,  0.,  0.,  0.],
       [ 0., 78., 79., ...,  0.,  0.,  0.],
       [ 0., 50., 50., ...,  0.,  0.,  0.],
       ...,
       [ 0., 80., 77., ...,  0.,  0.,  0.],
       [67., 69., 67., ...,  0.,  0.,  0.],
       [62., 63., 63., ...,  0.,  0.,  0.]])

In [26]:
X3_test_ohe =  X3_test_ohe.to_numpy()
X3_test_ohe

array([[ 0.,  0., 76., ...,  0.,  0.,  0.],
       [ 0., 53., 53., ...,  0.,  0.,  0.],
       [ 0., 91., 91., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [52., 52., 52., ...,  0.,  0.,  0.],
       [ 0., 57., 58., ...,  0.,  0.,  0.]])

In [27]:
X3_train_label =  X3_train_label.to_numpy()
X3_test_label =  X3_test_label.to_numpy()

input시 전달 ndarray는 3차원이여야 한다.

In [28]:
X3_train_ohe = X3_train_ohe.reshape(X3_train_ohe.shape[0],X3_train_ohe.shape[1],1)
X3_test_ohe = X3_test_ohe.reshape(X3_test_ohe.shape[0],X3_test_ohe.shape[1],1)

In [29]:
X3_train_label = X3_train_label.reshape(X3_train_label.shape[0],X3_train_label.shape[1],1)
X3_test_label = X3_test_label.reshape(X3_test_label.shape[0],X3_test_label.shape[1],1)

In [30]:
# hyperparameters
units = 50
timesteps = 1
inputs = (X3_train_ohe.shape[1],X3_train_ohe.shape[2])
outputs = 3
epochs = 50
batch_size = 256
# early_stop 하기 위해 관찰 값 monitor
# 관찰 값이 loss 이므로 최소화 시키는 방향으로 training이 진행 (default 깂 Auto)
# patience : 성능이 증가하지 않더라도 epoch을 얼마나 더 진행해볼 것인가?
early_stopping = EarlyStopping(monitor='val_loss', mode = 'min'
                               , patience=10, verbose=1)

### 기본 RNN 모델

In [31]:
def vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(units, input_shape = inputs, return_sequences = False)) 
    model.add(Dense(outputs)) #target 개수 = 3
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    #y를 one-hot encoding 시 'categorical_crossentropy 사용
    #y를 label encoding 시 'sparse_categorical_crossentropy' 사용
    model.compile(loss='categorical_crossentropy', optimizer = adam, metrics = ['accuracy']) 
    
    return model

### Single Layer one-hot encoding으로 fit

In [32]:
signle_rnn_model = KerasClassifier(build_fn = vanilla_rnn
                        , epochs = epochs, batch_size = batch_size, verbose = 1
                       , validation_split = 0.2, callbacks=[early_stopping])
signle_rnn_model.fit(X3_train_ohe, y3_train_ohe)
y3_pred_rnn_ohe = signle_rnn_model.predict(X3_test_ohe)

Train on 5968 samples, validate on 1493 samples
Epoch 1/50
5968/5968 [==============================] - 1s 151us/sample - loss: 0.9079 - acc: 0.5938 - val_loss: 0.8437 - val_acc: 0.6430
Epoch 2/50
5968/5968 [==============================] - 1s 131us/sample - loss: 1.0939 - acc: 0.4228 - val_loss: 1.3557 - val_acc: 0.1038
Epoch 3/50
5968/5968 [==============================] - 1s 135us/sample - loss: 0.8696 - acc: 0.6128 - val_loss: 0.8401 - val_acc: 0.6443
Epoch 4/50
5968/5968 [==============================] - 1s 132us/sample - loss: 0.8362 - acc: 0.6366 - val_loss: 0.8295 - val_acc: 0.6443
Epoch 5/50
5968/5968 [==============================] - 1s 129us/sample - loss: 0.8296 - acc: 0.6364 - val_loss: 0.8290 - val_acc: 0.6443
Epoch 6/50
5968/5968 [==============================] - 1s 130us/sample - loss: 0.8284 - acc: 0.6366 - val_loss: 0.8275 - val_acc: 0.6443
Epoch 7/50
5968/5968 [==============================] - 1s 132us/sample - loss: 0.8287 - acc: 0.6362 - val_loss: 0.8278 - va

In [33]:
print("RNN accouracy : ", accuracy_score(y3_pred_rnn_ohe,y3_test_label))

RNN accouracy :  0.6382636655948553


### Single Layer label encoding으로 fit

In [34]:
signle_rnn_model.fit(X3_train_label, y3_train_label)
y3_pred_rnn_label = signle_rnn_model.predict(X3_test_label)

Train on 5968 samples, validate on 1493 samples
Epoch 1/50
5968/5968 [==============================] - 1s 151us/sample - loss: 0.8884 - acc: 0.6136 - val_loss: 0.8258 - val_acc: 0.6450
Epoch 2/50
5968/5968 [==============================] - 1s 135us/sample - loss: 0.8277 - acc: 0.6357 - val_loss: 0.8207 - val_acc: 0.6450
Epoch 3/50
5968/5968 [==============================] - 1s 135us/sample - loss: 0.8236 - acc: 0.6354 - val_loss: 0.8215 - val_acc: 0.6450
Epoch 4/50
5968/5968 [==============================] - 1s 135us/sample - loss: 0.8250 - acc: 0.6361 - val_loss: 0.8169 - val_acc: 0.6470
Epoch 5/50
5968/5968 [==============================] - 1s 137us/sample - loss: 0.8193 - acc: 0.6366 - val_loss: 0.8090 - val_acc: 0.6470
Epoch 6/50
5968/5968 [==============================] - 1s 133us/sample - loss: 0.8177 - acc: 0.6337 - val_loss: 0.8107 - val_acc: 0.6463
Epoch 7/50
5968/5968 [==============================] - 1s 133us/sample - loss: 0.8214 - acc: 0.6369 - val_loss: 0.8168 - va

In [35]:
print("RNN accouracy : ", accuracy_score(y3_pred_rnn_label,y3_test_label))

RNN accouracy :  0.6393354769560557


### 다층 RNN 모델

In [36]:
def stacked_vanilla_rnn():
    model = Sequential()
    # return_sequences parameter has to be set True to stack
    model.add(SimpleRNN(units, input_shape = inputs, return_sequences = True))   
    model.add(SimpleRNN(units, return_sequences = False))
    model.add(Dense(outputs))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss='categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [37]:
multi_rnn_model = KerasClassifier(build_fn = stacked_vanilla_rnn
                        , epochs = epochs, batch_size = batch_size, verbose = 1
                       , validation_split = 0.2, callbacks=[early_stopping])
multi_rnn_model.fit(X3_train_ohe, y3_train_ohe)
y3_pred_multi_rnn_ohe = multi_rnn_model.predict(X3_test_ohe)

Train on 5968 samples, validate on 1493 samples
Epoch 1/50
5968/5968 [==============================] - 2s 345us/sample - loss: 0.8651 - acc: 0.5992 - val_loss: 0.8523 - val_acc: 0.6437
Epoch 2/50
5968/5968 [==============================] - 2s 321us/sample - loss: 0.8337 - acc: 0.6367 - val_loss: 0.8321 - val_acc: 0.6437
Epoch 3/50
5968/5968 [==============================] - 2s 326us/sample - loss: 0.8249 - acc: 0.6359 - val_loss: 0.8176 - val_acc: 0.6457
Epoch 4/50
5968/5968 [==============================] - 2s 334us/sample - loss: 0.8199 - acc: 0.6369 - val_loss: 0.8215 - val_acc: 0.6457
Epoch 5/50
5968/5968 [==============================] - 2s 327us/sample - loss: 0.8178 - acc: 0.6387 - val_loss: 0.8122 - val_acc: 0.6457
Epoch 6/50
5968/5968 [==============================] - 2s 327us/sample - loss: 0.8186 - acc: 0.6386 - val_loss: 0.8110 - val_acc: 0.6477
Epoch 7/50
5968/5968 [==============================] - 2s 322us/sample - loss: 0.8141 - acc: 0.6387 - val_loss: 0.8021 - va

In [38]:
print("accouracy : ", accuracy_score(y3_pred_multi_rnn_ohe,y3_test_label))

accouracy :  0.6366559485530546


### Multi Layer RNN (Label Encoding)

In [ ]:
multi_rnn_model.fit(X3_train_label, y3_train_label)
y3_pred_multi_rnn_label = multi_rnn_model.predict(X3_test_label)

Train on 5968 samples, validate on 1493 samples
Epoch 1/50
1280/5968 [=====>........................] - ETA: 1s - loss: 0.9453 - acc: 0.5430

In [ ]:
print("accouracy : ", accuracy_score(y3_pred_multi_rnn_label,y3_test_label))

### 단층 LSTM 모델

In [ ]:
def lstm():
    model = Sequential()
    model.add(LSTM(units, input_shape = inputs, return_sequences = False))
    model.add(Dense(outputs))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss='categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

### Single Layer LSTM (one-hot encoding)

In [ ]:
single_lstm_model = KerasClassifier(build_fn = lstm
                        , epochs = epochs, batch_size = batch_size, verbose = 1
                       , validation_split = 0.2, callbacks=[early_stopping])
single_lstm_model.fit(X3_train_ohe, y3_train_ohe)
y3_pred_single_lstm = single_lstm_model.predict(X3_test_ohe)

In [ ]:
print("accouracy : ", accuracy_score(y3_pred_single_lstm,y3_test_label))

### Single Layer LSTM (Label encoding)

In [ ]:
single_lstm_model.fit(X3_train_label, y3_train_label)
y3_pred_single_lstm_label = single_lstm_model.predict(X3_test_label)

In [ ]:
print("accouracy : ", accuracy_score(y3_pred_single_lstm_label,y3_test_label))

## Multi Layer LSTM 모델 구축

In [ ]:
def stacked_lstm():
    model = Sequential()
    model.add(LSTM(units, input_shape = inputs, return_sequences = True))
    model.add(LSTM(units, return_sequences = False))
    model.add(Dense(outputs))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss='categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

### Multi Layer LSTM (one-hot encoding)

In [ ]:
multi_lstm_model = KerasClassifier(build_fn = stacked_lstm
                        , epochs = epochs, batch_size = batch_size, verbose = 1
                       , validation_split = 0.2, callbacks=[early_stopping])
multi_lstm_model.fit(X3_train_ohe, y3_train_ohe)
y3_pred_multi_lstm = multi_lstm_model.predict(X3_test_ohe)

In [ ]:
print("LSTM multi  layer accouracy : ", accuracy_score(y3_pred_multi_lstm,y3_test_label))

### Multi Layer LSTM (Label encoding)

In [ ]:
multi_lstm_model.fit(X3_train_label, y3_train_label)
y3_pred_multi_lstm_label = multi_lstm_model.predict(X3_test_label)

In [ ]:
print("LSTM multi  layer accouracy : ", accuracy_score(y3_pred_multi_lstm_label,y3_test_label))

----------------------------------------------------------------------------

one-hot encoding 결과 비교

In [ ]:
print("RNN single layer accouracy : ", accuracy_score(y3_pred_rnn_ohe,y3_test_label))
print("RNN multi  layer accouracy : ", accuracy_score(y3_pred_multi_rnn_ohe,y3_test_label))
print("LSTM single layer accouracy : ", accuracy_score(y3_pred_single_lstm,y3_test_label))
print("LSTM multi  layer accouracy : ", accuracy_score(y3_pred_multi_lstm,y3_test_label))

label encoding 결과 비교

In [ ]:
print("RNN single layer accouracy : ", accuracy_score(y3_pred_rnn_label,y3_test_label))
print("RNN multi  layer accouracy : ", accuracy_score(y3_pred_multi_rnn_label,y3_test_label))
print("LSTM single layer accouracy : ", accuracy_score(y3_pred_single_lstm_label,y3_test_label))
print("LSTM multi  layer accouracy : ", accuracy_score(y3_pred_multi_lstm_label,y3_test_label))

-----------------------------------------------------------------------------

# 형빈 확인

일단 X3 : signal_sleep_hr_5min 에 대해서만 결과 출력  
모델 구축 다 했으니까 각각 변수 넣어서 결과 출력하는건 얼마 안걸림  
---------지금은 1. 모델에 따른 결과 차이 2. 인코딩 방식에 따른 결과 차이 확인 ------------
